# 🎬 MCP Server Demo — From REST API to Model Context Protocol

## Step-by-step client demo

This notebook demonstrates how Azure API Management transforms REST APIs into **Model Context Protocol (MCP)** servers — and how new MCP servers become **automatically discoverable** in Azure API Center.

### Demo story
1. **Deploy 3 MCP servers** (Weather, Product Catalog, Place Order) from existing REST APIs
2. **Verify** they are registered and discoverable in API Center
3. **Test** each MCP server with direct HTTP calls
4. **Add a 4th MCP server** (Calculator) as an add-on deployment
5. **Show** it is immediately discoverable in API Center without any manual registration

### Prerequisites
- [Azure CLI](https://learn.microsoft.com/cli/azure/install-azure-cli) installed and signed in
- [Python 3.12+](https://www.python.org/) with the project requirements installed
- An Azure subscription with **Contributor** access

---
## Part 1 — Deploy Infrastructure + 3 MCP Servers
---

### 0️⃣ Initialize

In [7]:
import os, sys, json
sys.path.insert(1, '../../shared')
import utils

deployment_name = "mcp-demo-1"
resource_group_name = f"rg-lab-{deployment_name}-1"
resource_group_location = "uksouth"

apim_sku = "Basicv2"
apim_name = "apim-mcp-demo-1"
apim_subscriptions_config = [{"name": "subscription1", "displayName": "Subscription 1"}]

apic_location = "uksouth"
apic_service_name_prefix = "apic-demo-1"

utils.print_ok("Variables initialized")
print(f"  Resource Group: {resource_group_name}")
print(f"  APIM SKU:       {apim_sku}")
print(f"  Location:       {resource_group_location}")

✅ Variables initialized ⌚ 13:28:52.448877 
  Resource Group: rg-lab-mcp-demo-1-1
  APIM SKU:       Basicv2
  Location:       uksouth


### 1️⃣ Deploy infrastructure + 3 MCP servers using 🦾 Bicep

Deploys in a single Bicep template:
| Layer | Resources |
|-------|-----------|
| **Monitoring** | Log Analytics, Application Insights |
| **API Gateway** | API Management (Basicv2) |
| **API Governance** | API Center |
| **Weather** | REST API → MCP Server (no auth) |
| **Product Catalog** | REST API → MCP Server (JWT auth) |
| **Place Order** | REST API + Logic App → MCP Server (JWT auth) |

> ⏱️ First deployment takes ~5-8 minutes (APIM provisioning). Subsequent runs are incremental.

In [8]:
# Create the resource group
utils.create_resource_group(resource_group_name, resource_group_location)

# Build Bicep parameters
bicep_parameters = {
    "$schema": "https://schema.management.azure.com/schemas/2019-04-01/deploymentParameters.json#",
    "contentVersion": "1.0.0.0",
    "parameters": {
        "apimSku": { "value": apim_sku },
        "apimName": { "value": apim_name },
        "apimSubscriptionsConfig": { "value": apim_subscriptions_config },
        "apicLocation": { "value": apic_location },
        "apicServiceNamePrefix": { "value": apic_service_name_prefix }
    }
}

with open('params-demo.json', 'w') as f:
    f.write(json.dumps(bicep_parameters))

output = utils.run(
    f"az deployment group create --name {deployment_name} --resource-group {resource_group_name} --template-file demo-initial.bicep --parameters params-demo.json",
    f"✅ Deployment '{deployment_name}' succeeded",
    f"❌ Deployment '{deployment_name}' failed"
)

⚙️ Running: az group show --name rg-lab-mcp-demo-1-1 
👉🏽 Resource group rg-lab-mcp-demo-1-1 does not yet exist. Creating the resource group now...
⚙️ Running: az group create --name rg-lab-mcp-demo-1-1 --location uksouth --tags source=ai-gateway 
✅ Resource group 'rg-lab-mcp-demo-1-1' created ⌚ 13:29:18.524228 :6s]
⚙️ Running: az deployment group create --name mcp-demo-1 --resource-group rg-lab-mcp-demo-1-1 --template-file demo-initial.bicep --parameters params-demo.json 
✅ ✅ Deployment 'mcp-demo-1' succeeded ⌚ 13:32:18.478148 :59s]


### 2️⃣ Retrieve deployment outputs

In [9]:
output = utils.run(f"az deployment group show --name {deployment_name} -g {resource_group_name}",
                   f"Retrieved deployment outputs",
                   f"Failed to retrieve deployment")

if output.success and output.json_data:
    apim_service_name = utils.get_deployment_output(output, 'apimServiceName', 'APIM Service Name')
    apim_resource_gateway_url = utils.get_deployment_output(output, 'apimResourceGatewayURL', 'APIM Gateway URL')
    apic_service_name = utils.get_deployment_output(output, 'apicServiceName', 'API Center Name')
    apic_api_env = utils.get_deployment_output(output, 'apicApiEnvironmentName', 'API Center API Environment')
    apic_mcp_env = utils.get_deployment_output(output, 'apicMcpEnvironmentName', 'API Center MCP Environment')

    apim_subscriptions = json.loads(utils.get_deployment_output(output, 'apimSubscriptions').replace("\'", "\""))
    api_key = apim_subscriptions[0].get("key")
    utils.print_info(f"API Key: ****{api_key[-4:]}")

    weather_mcp_endpoint = utils.get_deployment_output(output, 'weatherMCPEndpoint', 'Weather MCP')
    catalog_mcp_endpoint = utils.get_deployment_output(output, 'productCatalogMCPEndpoint', 'Catalog MCP')
    order_mcp_endpoint = utils.get_deployment_output(output, 'placeOrderMCPEndpoint', 'Place Order MCP')

⚙️ Running: az deployment group show --name mcp-demo-1 -g rg-lab-mcp-demo-1-1 
✅ Retrieved deployment outputs ⌚ 13:32:30.589432 :7s]
👉🏽 APIM Service Name: apim-mcp-demo-1
👉🏽 APIM Gateway URL: https://apim-mcp-demo-1.azure-api.net
👉🏽 API Center Name: apic-demo-1-xbilmxrmx74wu
👉🏽 API Center API Environment: api
👉🏽 API Center MCP Environment: mcp
👉🏽 API Key: ****5f57
👉🏽 Weather MCP: https://apim-mcp-demo-1.azure-api.net/weather-mcp/mcp
👉🏽 Catalog MCP: https://apim-mcp-demo-1.azure-api.net/catalog-mcp/mcp
👉🏽 Place Order MCP: https://apim-mcp-demo-1.azure-api.net/order-mcp/mcp


---
## Part 2 — Verify Discoverability in API Center
---

### 3️⃣ List all APIs in API Center

After deployment, both the REST APIs **and** MCP servers are automatically registered in API Center.  
Notice the `kind` column — `rest` for APIs, `mcp` for MCP servers.

In [10]:
output = utils.run(
    f'az apic api list -g {resource_group_name} -n {apic_service_name} --query "[].{{Name:name, Title:title, Kind:kind}}" -o table',
    "Listed APIs in API Center", "Failed to list APIs")
if output.success:
    print(output.text)

# Count and snapshot by kind
output = utils.run(
    f'az apic api list -g {resource_group_name} -n {apic_service_name} -o json',
    "", "")
if output.success and output.json_data:
    apis_before = output.json_data  # snapshot for later comparison
    rest_count = sum(1 for api in apis_before if api.get('kind') == 'rest')
    mcp_count = sum(1 for api in apis_before if api.get('kind') == 'mcp')
    print(f"\n📊 Total: {len(apis_before)} APIs registered — {rest_count} REST APIs, {mcp_count} MCP Servers")

⚙️ Running: az apic api list -g rg-lab-mcp-demo-1-1 -n apic-demo-1-xbilmxrmx74wu --query "[].{Name:name, Title:title, Kind:kind}" -o table 
✅ Listed APIs in API Center ⌚ 13:32:43.700626 :6s]
Name              Title                Kind
----------------  -------------------  ------
sentry            Sentry               mcp
cloudflare        Cloudflare           mcp
atlassian         Atlassian            mcp
paypal            Paypal               mcp
linear            Linear               mcp
intercom          Intercom             mcp
square            Square               mcp
asana             Asana                mcp
plaid             Plaid                mcp
swagger-petstore  Swagger Petstore     rest
weather-api       Weather API          rest
order-api         Place Order API      rest
catalog-api       Product Catalog API  rest
order-mcp         Place Order MCP      mcp
weather-mcp       Weather MCP          mcp
catalog-mcp       Product Catalog MCP  mcp

⚙️ Running: az apic api li

### ✅ Validate: Initial deployment — 3 REST APIs + 3 MCP Servers

Verify that exactly the expected APIs are registered before adding the Calculator.

In [12]:
# Validation: check expected APIs are registered
expected_before = {
    "rest": {"weather-api", "catalog-api", "order-api"},
    "mcp":  {"weather-mcp", "catalog-mcp", "order-mcp"}
}

actual_rest = {api['name'] for api in apis_before if api.get('kind') == 'rest'}
actual_mcp  = {api['name'] for api in apis_before if api.get('kind') == 'mcp'}

print("🔍 Validation — Initial Deployment")
print("-" * 50)

# Check REST APIs
missing_rest = expected_before["rest"] - actual_rest
extra_rest = actual_rest - expected_before["rest"]
if not missing_rest:
    utils.print_ok(f"REST APIs: all {len(expected_before['rest'])} expected APIs found ✅")
else:
    utils.print_error(f"REST APIs: missing {missing_rest}")
if extra_rest:
    utils.print_info(f"  Additional REST APIs (from API Center defaults): {extra_rest}")

# Check MCP Servers
missing_mcp = expected_before["mcp"] - actual_mcp
extra_mcp = actual_mcp - expected_before["mcp"]
if not missing_mcp:
    utils.print_ok(f"MCP Servers: all {len(expected_before['mcp'])} expected servers found ✅")
else:
    utils.print_error(f"MCP Servers: missing {missing_mcp}")
if extra_mcp:
    utils.print_info(f"  Additional MCP Servers (from API Center defaults): {extra_mcp}")

# Confirm calculator is NOT yet present
if "calculator-api" not in actual_rest and "calculator-mcp" not in actual_mcp:
    utils.print_ok("Calculator is NOT yet registered — ready for add-on demo ✅")
else:
    utils.print_error("⚠️ Calculator already exists — re-deploy without calculator first for a clean demo")

🔍 Validation — Initial Deployment
--------------------------------------------------
✅ REST APIs: all 3 expected APIs found ✅ ⌚ 13:42:26.887064 
👉🏽   Additional REST APIs (from API Center defaults): {'swagger-petstore'}
✅ MCP Servers: all 3 expected servers found ✅ ⌚ 13:42:26.888075 
👉🏽   Additional MCP Servers (from API Center defaults): {'plaid', 'paypal', 'sentry', 'asana', 'linear', 'cloudflare', 'square', 'atlassian', 'intercom'}
✅ Calculator is NOT yet registered — ready for add-on demo ✅ ⌚ 13:42:26.888075 


---
## Part 3 — Test Each MCP Server
---

### 🧪 Test Weather MCP (no authentication required)

The Weather MCP wraps a REST API that returns mock weather data.  
No JWT token needed — this MCP is open access.

In [13]:
import requests, json

utils.print_info("Calling Weather MCP → get-weather(city='London')...")
request = {
    "method": "tools/call",
    "params": {"name": "get-weather", "arguments": {"city": "London"}},
    "jsonrpc": "2.0", "id": 1
}
response = requests.post(weather_mcp_endpoint, stream=True,
                         headers={"Content-Type": "application/json", "agent-id": "Demo"},
                         json=request)
if response.status_code == 200:
    utils.print_ok("Weather MCP: HTTP 200 ✅")
    for line in response.iter_lines(decode_unicode=True):
        if line:
            if line == 'event: close':
                break
            if line.startswith('data'):
                data = json.loads(line.strip()[5:])
                print(json.dumps(json.loads(data["result"]["content"][0]["text"]), indent=4))
else:
    utils.print_error(f"Weather MCP: HTTP {response.status_code}")
response.close()

👉🏽 Calling Weather MCP → get-weather(city='London')...
✅ Weather MCP: HTTP 200 ✅ ⌚ 13:42:47.104122 
{
    "city": "London",
    "temperature": 1.1,
    "temperature_format": "Celsius",
    "description": "Rainy",
    "humidity": 63,
    "wind_speed": 7.9
}


### 🧪 Test Product Catalog MCP (JWT authentication required)

The Product Catalog MCP requires a valid JWT token.  
First we show that an unauthenticated call gets **401**, then we call with a token.

In [14]:
import requests, json

# 1. Test WITHOUT auth → expect 401
utils.print_info("Calling Catalog MCP WITHOUT auth → expect 401...")
request = {
    "method": "tools/call",
    "params": {"name": "get-product-details", "arguments": {"category": "electronics"}},
    "jsonrpc": "2.0", "id": 1
}
response = requests.post(catalog_mcp_endpoint, stream=True,
                         headers={"Content-Type": "application/json"},
                         json=request)
if response.status_code == 401:
    utils.print_ok("Catalog MCP: 401 Unauthorized — JWT policy is working ✅")
else:
    utils.print_error(f"Expected 401 but got HTTP {response.status_code}")
response.close()

# 2. Test WITH auth → expect 200
utils.print_info("Calling Catalog MCP WITH auth → expect 200...")
output = utils.run('az account get-access-token --resource "https://azure-api.net/authorization-manager"')
if output.success and output.json_data:
    access_token = output.json_data['accessToken']
    response = requests.post(catalog_mcp_endpoint, stream=True,
                             headers={"Content-Type": "application/json", "agent-id": "Demo",
                                      "Authorization": f"Bearer {access_token}"},
                             json=request)
    if response.status_code == 200:
        utils.print_ok("Catalog MCP: HTTP 200 with auth ✅")
        for line in response.iter_lines(decode_unicode=True):
            if line:
                if line == 'event: close':
                    break
                if line.startswith('data'):
                    data = json.loads(line.strip()[5:])
                    print(json.dumps(json.loads(data["result"]["content"][0]["text"]), indent=4))
    else:
        utils.print_error(f"Catalog MCP: HTTP {response.status_code}")
    response.close()

👉🏽 Calling Catalog MCP WITHOUT auth → expect 401...
✅ Catalog MCP: 401 Unauthorized — JWT policy is working ✅ ⌚ 13:46:28.566467 
👉🏽 Calling Catalog MCP WITH auth → expect 200...
⚙️ Running: az account get-access-token --resource "https://azure-api.net/authorization-manager" 
✅ Catalog MCP: HTTP 200 with auth ✅ ⌚ 13:46:43.474562 
{
    "name": "Laptop",
    "category": "electronics",
    "sku": "SKU-1234",
    "stock": 40,
    "store_location": "London"
}


### 🧪 Test Place Order MCP (JWT authentication required)

The Place Order MCP wraps a Logic App workflow. It also requires JWT authentication.

In [15]:
import requests, json

utils.print_info("Calling Place Order MCP → PlaceOrder-invoke(sku='SKU-1234', quantity=2)...")
request = {
    "method": "tools/call",
    "params": {"name": "PlaceOrder-invoke", "arguments": {"sku": "SKU-1234", "quantity": 2}},
    "jsonrpc": "2.0", "id": 1
}

# Get auth token
output = utils.run('az account get-access-token --resource "https://azure-api.net/authorization-manager"')
if output.success and output.json_data:
    access_token = output.json_data['accessToken']
    response = requests.post(order_mcp_endpoint, stream=True,
                             headers={"Content-Type": "application/json", "agent-id": "Demo",
                                      "Authorization": f"Bearer {access_token}"},
                             json=request)
    if response.status_code == 200:
        utils.print_ok("Place Order MCP: HTTP 200 ✅")
        for line in response.iter_lines(decode_unicode=True):
            if line:
                if line == 'event: close':
                    break
                if line.startswith('data'):
                    data = json.loads(line.strip()[5:])
                    print(json.dumps(json.loads(data["result"]["content"][0]["text"]), indent=4))
    else:
        utils.print_error(f"Place Order MCP: HTTP {response.status_code} - {response.text[:200]}")
    response.close()

👉🏽 Calling Place Order MCP → PlaceOrder-invoke(sku='SKU-1234', quantity=2)...
⚙️ Running: az account get-access-token --resource "https://azure-api.net/authorization-manager" 
✅ Place Order MCP: HTTP 200 ✅ ⌚ 13:47:02.410192 
{
    "error": {
        "code": "DirectApiRequestHasMoreThanOneAuthorization",
        "message": "The request has SAS authentication scheme and an additional authorization scheme or internal token scheme. Only one scheme should be used."
    }
}


---
## Part 4 — Add a New MCP Server and Show Auto-Discovery
---

> 💡 **This is the key demo moment.**  
> We deploy a **4th MCP server** (Calculator) into the same APIM + API Center.  
> After deployment, it **automatically appears** in API Center — no manual registration needed.

### 4️⃣ Deploy Calculator MCP as an add-on

This uses a separate Bicep file that targets the **existing** APIM and API Center resources.  
It deploys the Calculator REST API + its MCP server.

In [16]:
calculator_deployment_name = f"{deployment_name}-calculator"

bicep_parameters = {
    "$schema": "https://schema.management.azure.com/schemas/2019-04-01/deploymentParameters.json#",
    "contentVersion": "1.0.0.0",
    "parameters": {
        "apimServiceName": { "value": apim_service_name },
        "apicServiceName": { "value": apic_service_name },
        "apicApiEnvironmentName": { "value": apic_api_env },
        "apicMcpEnvironmentName": { "value": apic_mcp_env }
    }
}

with open('params-demo-calculator.json', 'w') as f:
    f.write(json.dumps(bicep_parameters))

output = utils.run(
    f"az deployment group create --name {calculator_deployment_name} --resource-group {resource_group_name} --template-file demo-add-calculator.bicep --parameters params-demo-calculator.json",
    f"✅ Calculator MCP deployed!",
    f"❌ Calculator deployment failed"
)

if output.success and output.json_data:
    calculator_mcp_endpoint = output.json_data['properties']['outputs']['calculatorMCPEndpoint']['value']
    utils.print_info(f"Calculator MCP Endpoint: {calculator_mcp_endpoint}")

⚙️ Running: az deployment group create --name mcp-demo-1-calculator --resource-group rg-lab-mcp-demo-1-1 --template-file demo-add-calculator.bicep --parameters params-demo-calculator.json 
✅ ✅ Calculator MCP deployed! ⌚ 13:48:09.539286 :50s]


### 5️⃣ 🔍 Verify auto-discovery — Calculator now appears in API Center!

Compare this output with the earlier API Center listing.  
You will see **2 new entries**: `calculator-api` (rest) and `calculator-mcp` (mcp).

In [17]:
output = utils.run(
    f'az apic api list -g {resource_group_name} -n {apic_service_name} --query "[].{{Name:name, Title:title, Kind:kind}}" -o table',
    "Listed APIs in API Center", "Failed to list APIs")
if output.success:
    print(output.text)

# Count and snapshot
output = utils.run(
    f'az apic api list -g {resource_group_name} -n {apic_service_name} -o json',
    "", "")
if output.success and output.json_data:
    apis_after = output.json_data
    rest_after = sum(1 for api in apis_after if api.get('kind') == 'rest')
    mcp_after = sum(1 for api in apis_after if api.get('kind') == 'mcp')
    print(f"\n📊 Total: {len(apis_after)} APIs registered — {rest_after} REST APIs, {mcp_after} MCP Servers")
    print(f"🆕 Calculator MCP is now discoverable!")

⚙️ Running: az apic api list -g rg-lab-mcp-demo-1-1 -n apic-demo-1-xbilmxrmx74wu --query "[].{Name:name, Title:title, Kind:kind}" -o table 
✅ Listed APIs in API Center ⌚ 13:48:21.693036 :5s]
Name              Title                Kind
----------------  -------------------  ------
sentry            Sentry               mcp
cloudflare        Cloudflare           mcp
atlassian         Atlassian            mcp
paypal            Paypal               mcp
linear            Linear               mcp
intercom          Intercom             mcp
square            Square               mcp
asana             Asana                mcp
plaid             Plaid                mcp
swagger-petstore  Swagger Petstore     rest
weather-api       Weather API          rest
order-api         Place Order API      rest
catalog-api       Product Catalog API  rest
order-mcp         Place Order MCP      mcp
weather-mcp       Weather MCP          mcp
catalog-mcp       Product Catalog MCP  mcp
calculator-api    Calculato

### ✅ Validate: Before vs After — Auto-discovery proof

Compare the API Center snapshots taken before and after adding the Calculator MCP.

In [18]:
# Before vs After comparison
names_before = {api['name'] for api in apis_before}
names_after  = {api['name'] for api in apis_after}
new_apis = names_after - names_before

print("🔍 Validation — Before vs After Comparison")
print("=" * 55)

rest_before = sum(1 for api in apis_before if api.get('kind') == 'rest')
mcp_before  = sum(1 for api in apis_before if api.get('kind') == 'mcp')
rest_after  = sum(1 for api in apis_after if api.get('kind') == 'rest')
mcp_after   = sum(1 for api in apis_after if api.get('kind') == 'mcp')

print(f"  {'':20} {'BEFORE':>10} {'AFTER':>10} {'DIFF':>10}")
print(f"  {'-'*20} {'-'*10} {'-'*10} {'-'*10}")
print(f"  {'REST APIs':20} {rest_before:>10} {rest_after:>10} {'+' + str(rest_after - rest_before):>10}")
print(f"  {'MCP Servers':20} {mcp_before:>10} {mcp_after:>10} {'+' + str(mcp_after - mcp_before):>10}")
print(f"  {'Total':20} {len(apis_before):>10} {len(apis_after):>10} {'+' + str(len(apis_after) - len(apis_before)):>10}")
print()

if new_apis:
    print(f"  🆕 Newly discovered APIs:")
    for name in sorted(new_apis):
        kind = next((api['kind'] for api in apis_after if api['name'] == name), '?')
        print(f"     • {name} ({kind})")
    print()

# Final assertions
all_passed = True
if "calculator-api" in names_after and "calculator-mcp" in names_after:
    utils.print_ok("Calculator API + MCP auto-discovered in API Center ✅")
else:
    utils.print_error("Calculator not found in API Center ❌")
    all_passed = False

if rest_after == rest_before + 1:
    utils.print_ok(f"REST API count increased by 1 ({rest_before} → {rest_after}) ✅")
else:
    utils.print_error(f"Unexpected REST count: {rest_before} → {rest_after}")
    all_passed = False

if mcp_after == mcp_before + 1:
    utils.print_ok(f"MCP Server count increased by 1 ({mcp_before} → {mcp_after}) ✅")
else:
    utils.print_error(f"Unexpected MCP count: {mcp_before} → {mcp_after}")
    all_passed = False

if all_passed:
    print()
    utils.print_ok("🎉 Auto-discovery validated — new MCP servers appear automatically!")

🔍 Validation — Before vs After Comparison
                           BEFORE      AFTER       DIFF
  -------------------- ---------- ---------- ----------
  REST APIs                     4          5         +1
  MCP Servers                  12         13         +1
  Total                        16         18         +2

  🆕 Newly discovered APIs:
     • calculator-api (rest)
     • calculator-mcp (mcp)

✅ Calculator API + MCP auto-discovered in API Center ✅ ⌚ 13:49:55.992104 
✅ REST API count increased by 1 (4 → 5) ✅ ⌚ 13:49:55.992104 
✅ MCP Server count increased by 1 (12 → 13) ✅ ⌚ 13:49:55.992104 

✅ 🎉 Auto-discovery validated — new MCP servers appear automatically! ⌚ 13:49:55.992104 


---
## Part 5 — Test Calculator MCP
---

### 🧪 Test Calculator MCP — All 4 operations

The Calculator MCP supports: `add`, `subtract`, `multiply`, `divide`.  
No authentication required.

In [ ]:
import requests, json

calculator_mcp_endpoint =''
test_cases = [
    ("add",      42, 17, 59),
    ("subtract", 100, 37, 63),
    ("multiply", 7,  8,  56),
    ("divide",   144, 12, 12),
]

all_passed = True
for op, a, b, expected in test_cases:
    request = {
        "method": "tools/call",
        "params": {"name": "calculate", "arguments": {"operation": op, "a": a, "b": b}},
        "jsonrpc": "2.0", "id": 1
    }
    response = requests.post(calculator_mcp_endpoint, stream=True,
                             headers={"Content-Type": "application/json", "agent-id": "Demo"},
                             json=request)
    if response.status_code == 200:
        for line in response.iter_lines(decode_unicode=True):
            if line:
                if line == 'event: close':
                    break
                if line.startswith('data'):
                    data = json.loads(line.strip()[5:])
                    result = json.loads(data["result"]["content"][0]["text"])
                    actual = result.get("result")
                    if actual == expected:
                        utils.print_ok(f"  {op}({a}, {b}) = {expected} ✅")
                    else:
                        utils.print_error(f"  {op}({a}, {b}) = {actual} — expected {expected} ❌")
                        all_passed = False
    else:
        utils.print_error(f"  {op}: HTTP {response.status_code}")
        all_passed = False
    response.close()

if all_passed:
    utils.print_ok("\n🎉 All Calculator MCP operations verified!")

NameError: name 'calculator_mcp_endpoint' is not defined

---
## ✅ Demo Summary
---

In [ ]:
print("=" * 64)
print("  🎬  DEMO SUMMARY")
print("=" * 64)
print(f"  Resource Group:  {resource_group_name}")
print(f"  APIM Gateway:    {apim_resource_gateway_url}")
print(f"  API Center:      {apic_service_name}")
print()
print("  MCP Servers Deployed:")
print(f"    1. Weather MCP:        {weather_mcp_endpoint}")
print(f"    2. Catalog MCP:        {catalog_mcp_endpoint}")
print(f"    3. Place Order MCP:    {order_mcp_endpoint}")
print(f"    4. Calculator MCP:     {calculator_mcp_endpoint}  🆕")
print()
print("  Key takeaways:")
print("    • REST APIs transformed to MCP with zero code changes")
print("    • New MCP servers auto-discovered in API Center")
print("    • JWT auth policies enforce security at the gateway")
print("    • Full observability via App Insights tracing")
print("=" * 64)

---
## 🗑️ Clean up resources

Uncomment and run the cell below to delete all demo resources.

In [ ]:
# Uncomment to delete all demo resources:
# utils.run(f"az group delete --name {resource_group_name} --yes --no-wait",
#           f"Resource group '{resource_group_name}' deletion initiated",
#           f"Failed to delete resource group")